<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-chatha-bit/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Sachman Chatha

**Student Number**: 400246047

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [2]:
import pandas as pd

df = pd.read_csv('/content/natasupermarkets.csv')
df.isnull().sum()
df.dropna(inplace=True)
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed', dayfirst=True)

current_date = pd.Timestamp.now()
time_diff = current_date - df['Dt_Customer']
df['Days_With_Company'] = time_diff.dt.days
df.drop(columns=['Dt_Customer'], inplace=True)

df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Days_With_Company
0,5524,1957,Graduation,Single,58138.0,0,0,58,635,88,...,0,0,0,0,0,0,3,11,1,4978
1,2174,1954,Graduation,Single,46344.0,1,1,38,11,1,...,0,0,0,0,0,0,3,11,0,4132
2,4141,1965,Graduation,Together,71613.0,0,0,26,426,49,...,0,0,0,0,0,0,3,11,0,4479
3,6182,1984,Graduation,Together,26646.0,1,0,26,11,4,...,0,0,0,0,0,0,3,11,0,4072
4,5324,1981,PhD,Married,58293.0,1,0,94,173,43,...,0,0,0,0,0,0,3,11,0,4328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,46,709,43,...,0,0,0,0,0,0,3,11,0,4548
2236,4001,1946,PhD,Together,64014.0,2,1,56,406,0,...,0,0,0,1,0,0,3,11,0,4068
2237,7270,1981,Graduation,Divorced,56981.0,0,0,91,908,48,...,0,1,0,0,0,0,3,11,0,4322
2238,8235,1956,Master,Together,69245.0,0,1,8,428,30,...,0,0,0,0,0,0,3,11,0,4323


# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [3]:
y = df['MntWines']
X = df.drop(columns=['MntWines',
                     'ID',
                     'MntFruits',
                     'MntMeatProducts',
                      'MntFishProducts',
                      'MntSweetProducts',
                      'MntGoldProds',
                      'Response',
                      'Z_CostContact',
                      'Z_Revenue'])

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough')

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=20
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', rf_model)])

pipeline.fit(X_train, y_train)

print("Pipeline built and fitted successfully!")

Pipeline built and fitted successfully!


(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [6]:
import pandas as pd

new_customer_features = {
    'Education': 'Graduation',
    'Marital_Status': 'Married',
    'Income': 80000,
    'Kidhome': 1,
    'Teenhome': 1,
    'Recency': 43,
    'NumDealsPurchases': 2,
    'NumWebPurchases': 1,
    'NumCatalogPurchases': 0,
    'NumStorePurchases': 15,
    'NumWebVisitsMonth': 5,
    'AcceptedCmp1': 0,
    'AcceptedCmp2': 0,
    'AcceptedCmp3': 0,
    'AcceptedCmp4': 0,
    'AcceptedCmp5': 0,
    'Complain': 0}

new_customer_dt_customer_original = pd.to_datetime('2016-10-10', format='%Y-%m-%d')
new_customer_features['Year_Birth'] = new_customer_dt_customer_original.year - 48

derived_current_date = pd.Timestamp('2026-06-16')
new_customer_dt_customer = pd.to_datetime('2016-10-10', format='%Y-%m-%d')
new_customer_features['Days_With_Company'] = (derived_current_date - new_customer_dt_customer).days

expected_columns = list(X.columns)
new_customer_df = pd.DataFrame([new_customer_features])
new_customer_df = new_customer_df[expected_columns]

predicted_spending = pipeline.predict(new_customer_df)

print(f"Predicted spending on wines for the new customer: {predicted_spending[0]:.2f}")

Predicted spending on wines for the new customer: 389.56


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

Based on the 6-fold cross-validation results, the model's performance on the test dataset can be described as follows:

Mean R-squared Score: 0.8080 - This indicates that approximately 80.8% of the variance in customer wine spending in the test data can be explained by the features used in our model. An R-squared value close to 1 suggests a strong linear relationship between the predictors and the response variable, meaning the model is quite effective at capturing the patterns in wine spending.

Standard Deviation of R-squared Scores: 0.0248 - This low standard deviation suggests that the model's performance is consistent across different folds (subsets) of the data. This implies that the model is robust and not overly sensitive to the particular split of training and testing data, which is a desirable characteristic for a predictive model.

In conclusion, the model demonstrates good predictive accuracy and reliability on unseen data, as evidenced by a high mean R-squared and consistent performance across different test sets.

(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

cv_scores = cross_val_score(pipeline, X, y, cv=6, scoring='r2', n_jobs=-1)

print(f"Cross-validation R-squared scores: {cv_scores}")
print(f"Mean R-squared score: {np.mean(cv_scores):.4f}")
print(f"Standard deviation of R-squared scores: {np.std(cv_scores):.4f}")

Cross-validation R-squared scores: [0.82991295 0.80896469 0.77260863 0.82806993 0.77671567 0.83182129]
Mean R-squared score: 0.8080
Standard deviation of R-squared scores: 0.0248


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [5, 10, 15],
    'regressor__min_samples_split': [3, 6],
    'regressor__min_samples_leaf': [2, 4, 8]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=6, scoring='r2', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_rf_model = grid_search.best_estimator_

print(f"Best Hyperparameter Combination: {best_params}")
print(f"Corresponding Best R-squared Score: {best_score:.4f}")
print("Best Random Forest Model retrieved successfully!")

Fitting 6 folds for each of 36 candidates, totalling 216 fits
Best Hyperparameter Combination: {'regressor__max_depth': 15, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 3, 'regressor__n_estimators': 100}
Corresponding Best R-squared Score: 0.8048
Best Random Forest Model retrieved successfully!


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

linear_model = LinearRegression()

linear_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', linear_model)])

linear_pipeline.fit(X_train, y_train)

y_pred_lr = linear_pipeline.predict(X_test)

r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print("\n--- Linear Regression Model Performance on Test Data ---")
print(f"R-squared (R2) Score (Linear Regression): {r2_lr:.4f}")
print(f"Mean Absolute Error (MAE) (Linear Regression): {mae_lr:.2f}")

y_pred_rf = best_rf_model.predict(X_test)

r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print("\n--- Best Random Forest Model Performance on Test Data ---")
print(f"R-squared (R2) Score (Best Random Forest): {r2_rf:.4f}")
print(f"Mean Absolute Error (MAE) (Best Random Forest): {mae_rf:.2f}")

print("\n--- Comparison ---")
if r2_rf > r2_lr:
    print(f"The Best Random Forest model has a higher R-squared ({r2_rf:.4f}) than Linear Regression ({r2_lr:.4f}), indicating better explanatory power.")
else:
    print(f"The Linear Regression model has a higher R-squared ({r2_lr:.4f}) than Best Random Forest ({r2_rf:.4f}), indicating better explanatory power.")

if mae_rf < mae_lr:
    print(f"The Best Random Forest model has a lower MAE ({mae_rf:.2f}) than Linear Regression ({mae_lr:.2f}), indicating smaller average prediction errors.")
else:
    print(f"The Linear Regression model has a lower MAE ({mae_lr:.2f}) than Best Random Forest ({mae_rf:.2f}), indicating smaller average prediction errors.")


--- Linear Regression Model Performance on Test Data ---
R-squared (R2) Score (Linear Regression): 0.6700
Mean Absolute Error (MAE) (Linear Regression): 119.71

--- Best Random Forest Model Performance on Test Data ---
R-squared (R2) Score (Best Random Forest): 0.7849
Mean Absolute Error (MAE) (Best Random Forest): 87.49

--- Comparison ---
The Best Random Forest model has a higher R-squared (0.7849) than Linear Regression (0.6700), indicating better explanatory power.
The Best Random Forest model has a lower MAE (87.49) than Linear Regression (119.71), indicating smaller average prediction errors.


(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd

abs_residuals_lr = np.abs(y_pred_lr - y_test)
abs_residuals_rf = np.abs(y_pred_rf - y_test)

residuals_df = pd.DataFrame({
    'Linear Regression Residuals': abs_residuals_lr,
    'Random Forest Residuals': abs_residuals_rf
})

fig = px.scatter(
    residuals_df,
    x='Linear Regression Residuals',
    y='Random Forest Residuals',
    title='Comparison of Absolute Residuals: Linear Regression vs. Random Forest',
    labels={
        'Linear Regression Residuals': '|y_pred_LR - y_test|',
        'Random Forest Residuals': '|y_pred_RF - y_test|'
    },
    hover_data={'Linear Regression Residuals': ':.2f', 'Random Forest Residuals': ':.2f'}
)

min_val = min(residuals_df['Linear Regression Residuals'].min(), residuals_df['Random Forest Residuals'].min())
max_val = max(residuals_df['Linear Regression Residuals'].max(), residuals_df['Random Forest Residuals'].max())

buffer = (max_val - min_val) * 0.05
min_plot_val = min_val - buffer
max_plot_val = max_val + buffer

fig.add_shape(
    type="line",
    x0=min_plot_val, y0=min_plot_val,
    x1=max_plot_val, y1=max_plot_val,
    line=dict(color="red", width=2, dash="dash"),
)

fig.update_layout(xaxis_range=[min_plot_val, max_plot_val], yaxis_range=[min_plot_val, max_plot_val])
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.show()

Interpretation:

- Points below the 45-degree line indicate instances where the Random Forest model had smaller absolute errors than the Linear Regression model.
- Points above the 45-degree line indicate instances where the Linear Regression model had smaller absolute errors than the Random Forest model.
- Points on the line indicate similar error magnitudes.

If most points are below the line, it suggests the Random Forest model generally makes more accurate predictions (smaller errors) than Linear Regression. This aligns with the R2 and MAE comparison if RF is indeed better.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [ ]:
meat_percentiles = df['MntMeatProducts'].quantile([1/3, 2/3])

lower_bound = meat_percentiles[1/3]
upper_bound = meat_percentiles[2/3]

print(f"33.33% Percentile for MntMeatProducts: {lower_bound:.2f}")
print(f"66.67% Percentile for MntMeatProducts: {upper_bound:.2f}")

def label_meat_spending(spending):
    if spending <= lower_bound:
        return 'low'
    elif spending >= upper_bound:
        return 'high'
    else:
        return 'medium'

df['Meat_Spending_Label'] = df['MntMeatProducts'].apply(label_meat_spending)

print("\nValue counts for Meat_Spending_Label:")
print(df['Meat_Spending_Label'].value_counts())

print("\nSample DataFrame with 'Meat_Spending_Label':")
print(df[['MntMeatProducts', 'Meat_Spending_Label']].head())

33.33% Percentile for MntMeatProducts: 24.00
66.67% Percentile for MntMeatProducts: 146.33

Value counts for Meat_Spending_Label:
Meat_Spending_Label
low       754
high      739
medium    723
Name: count, dtype: int64

Sample DataFrame with 'Meat_Spending_Label':
   MntMeatProducts Meat_Spending_Label
0              546                high
1                6                 low
2              127              medium
3               20                 low
4              118              medium


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

y_meat = df['Meat_Spending_Label']
X_meat = df.drop(columns=['MntMeatProducts', 'Meat_Spending_Label', 'ID', 'MntWines'])

X_train_meat, X_test_meat, y_train_meat, y_test_meat = train_test_split(X_meat, y_meat, test_size=0.2, random_state=20, stratify=y_meat)

categorical_features_meat = X_meat.select_dtypes(include=['object']).columns
numerical_features_meat = X_meat.select_dtypes(include=['int64', 'float64']).columns

preprocessor_meat = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_meat),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_meat)
    ],
    remainder='passthrough'
)

knn_model = KNeighborsClassifier(n_neighbors=5)

knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_meat),
    ('classifier', knn_model)
])

knn_pipeline.fit(X_train_meat, y_train_meat)

print("KNN model pipeline built and fitted successfully!")

KNN model pipeline built and fitted successfully!


(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [ ]:
from sklearn.metrics import classification_report

y_pred_meat = knn_pipeline.predict(X_test_meat)

report = classification_report(y_test_meat, y_pred_meat)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

        high       0.84      0.80      0.82       148
         low       0.76      0.87      0.81       151
      medium       0.65      0.58      0.61       145

    accuracy                           0.75       444
   macro avg       0.75      0.75      0.75       444
weighted avg       0.75      0.75      0.75       444



Interpretation of the Classification Report
Each row in the report corresponds to a class ('high', 'low', 'medium' spending).

*   **Precision**: The ability of the classifier not to label as positive a sample that is negative. For each class, it's the ratio of correctly predicted positive observations to the total predicted positive observations. A high precision means fewer false positives.
    - For example, if precision for 'high' is 0.70, it means that when the model predicts a customer as 'high' spender, it is correct 70% of the time.

*   **Recall (Sensitivity)**: The ability of the classifier to find all the positive samples. For each class, it's the ratio of correctly predicted positive observations to all observations in actual class. A high recall means fewer false negatives.
    - For example, if recall for 'high' is 0.65, it means that the model correctly identified 65% of all actual 'high' spenders.

*   **F1-Score**: The weighted average of Precision and Recall. It tries to find the balance between precision and recall. It is a good metric to use when you have uneven class distribution (though here it's relatively balanced).
    - A higher F1-score indicates better overall performance for that class, considering both precision and recall.

*   **Support**: The number of actual occurrences of the class in the specified dataset (here, the test dataset).
    - This tells us how many instances of each class were present in `y_test_meat`.

*   **Accuracy**: The overall accuracy of the model, which is the ratio of correctly predicted observations to the total observations.

*   **Macro Avg**: The average (unweighted) of precision, recall, and F1-score across all classes. It treats all classes equally.

*   **Weighted Avg**: The average of precision, recall, and F1-score across all classes, weighted by the support for each class. This is useful when class distributions are imbalanced.


# Describe how you used Gen. AI. in this assignment (2 points)

Whenever I had an error or was stuck, I used GenAI to help me move through the problem in an efficient way.

**Note**: The remaining 5 points will be assigned to readability of the work.